<a href="https://colab.research.google.com/github/kartikeyahl/MS_analysis/blob/main/PunchTimeAnalysis6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Importing the libraries
import numpy as np
import pandas as pd
from datetime import datetime
import math

In [49]:
# Importing the excel
!unzip punch_short_.xlsx
dataset = pd.read_excel('punch_short_.xlsx',parse_dates=['Date'],
    date_parser=lambda x: pd.to_datetime(x, format='%Y%m%d'))
dataset2=pd.read_excel('Master Data.xlsx')

Archive:  punch_short_.xlsx
replace [Content_Types].xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

KeyboardInterrupt: ignored

In [50]:
#Renaming column name to a valid column name format as per python
dataset.rename(columns={'IN/OUT': 'IN_OUT'}, inplace=True)

In [51]:
#Droping rows that conatain 1 in 'IN_OUT' column
dataset=dataset[dataset.IN_OUT!=1]

In [52]:
#Converting Date column values into year-month-day format
# j,k=0,0
# for i in dataset.iloc[:,3]:
#   dataset.Date[j]=datetime.strptime(str(i),'%Y%m%d').date()
#   j+=1

dataset['Date'] = pd.to_datetime(dataset['Date'], errors='coerce')

In [53]:
#Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)
l=[]
for i in range(len(dataset)-1):
  j=i+1
  x=str(dataset.iloc[i]['Time'])
  y=str(dataset.iloc[j]['Time'])

  if len(x)<=5:
    x=x.zfill(6)
  if len(y)<=5:
    y=y.zfill(6)

  x = datetime.strptime(x, "%H%M%S")
  x.strftime("%I:%M %p")
  y = datetime.strptime(y, "%H%M%S")
  y.strftime("%I:%M %p")

  l.append([dataset.iloc[i]['E.Code'], dataset.iloc[i]['Date'].date(), dataset.iloc[i]['IN_OUT'], int(dataset.iloc[i]['Time']), datetime.combine(dataset.iloc[j]['Date'], y.time()) - datetime.combine(dataset.iloc[i]['Date'], x.time())])  
  i=i+2

In [54]:
# #Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)

# x='140000'
# y='020000'
# if len(x)<=4:
#   x=x.zfill(6)
# if len(y)<=4:
#   y=y.zfill(6)

# x2 = datetime.strptime(x, "%H%M%S")
# x2.strftime("%I:%M %p")
# y2 = datetime.strptime(y, "%H%M%S")
# y2.strftime("%I:%M %p")
# print(x2,y2)
# r=datetime.combine(datetime.strptime('20221012','%Y%m%d'), y2.time()) - datetime.combine(datetime.strptime('20221011','%Y%m%d'), x2.time())
# print(r)

In [55]:
#Replacing ":" with "." and getting upto 2 decimal places in Time to make mathematical operations feasible on i
for i in range(len(l)):
  l[i][4] = l[i][4].__str__().replace(":",".")
  s=l[i][4]
  l[i][4]=s[0:-3]

In [56]:
#Removing redundant rows and storing in new data-structure
l2=[]
for i in range(len(l)):
  if 'P20' not in str(l[i][2]):
    l2.append(l[i]) 

In [57]:
for i in range(len(l2)):
  if 'd' in l2[i][4]:
    s=l2[i][4]
    s=float(s[8:len(s)])
    l2[i][4]=24+s

In [58]:
# s='1 day, 0.53'

# s=float(s[5:len(s)])
# s=24+s
# print(s)

In [59]:
#Converting string type to float for Time column
for i in range(len(l2)):
  try:
   l2[i][4]=float("{:.2f}".format(float(l2[i][4])))
  except:
    print(l2[i][0],l2[i][4])

In [60]:
"""Converting time from float to mins, applied logic to check if it is greater than 9hrs & 30mins. Subtacting the 
result to get over time (minutes) converting and storing it into hrs.minutes format. """

l3=[]
for i in range(len(l2)):
  
  frac, whole = math.modf(l2[i][4])
  mins=int(whole*60+frac*100)
  
  if l2[i][2]=='P10' and l2[i][3]>220000:
    # print(frac,whole)
    # print(l2[i][0])
    # print(l2[i][4])
    # print(mins)
    if mins>450:
      mins=mins-450
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  else:
    mins=mins-570
    hours=mins//60
    minutes=mins%60
    if minutes<10:
      ot_time = float("{}.0{}".format(hours, minutes))
    else:
      ot_time = float("{}.{}".format(hours, minutes))
    l2[i].append(ot_time)
    l3.append(l2[i])  

In [61]:
#Removing redundant rows and storing in new data-structure
l4=[]
for i in range(len(l3)):
  if l3[i][5]>0:
    l4.append(l3[i]) 

In [62]:
# Converting 2d list to dataframe to apply CRUD opertations
df = pd.DataFrame(l4, columns=['E.Code','Date','IN/OUT','Time','Total Time','OT'])

In [63]:
#Droping an unnecesary column
df=df.drop(['IN/OUT','Time','Total Time'], axis=1)

In [64]:
ll1=[]
def binarySearch(arr, l, r, x):

	while l <= r:

		mid = l + (r - l) // 2

		# Check if x is present at mid
		if arr[mid] == x:
			return mid

		# If x is greater, ignore left half
		elif arr[mid] < x:
			l = mid + 1

		# If x is smaller, ignore right half
		else:
			r = mid - 1

	# If we reach here, then the element
	# was not present
	return -1


# Driver Code
arr = [int(i) for i in dataset2['E Code']]

for i in range(len(df)-1):
  # Function call
  result = binarySearch(arr, 0, len(arr)-1, df.iloc[i]['E.Code'])

  if result != -1:
      ll1.append([dataset2.iloc[result]['Designation'],dataset2.iloc[result]['Operation'],dataset2.iloc[result]['Division'],dataset2.iloc[result]['Department']])
  else:
      ll1.append(['NaN','NaN','NaN','NaN'])


In [65]:
# ll1=[]
# def linearSearch(arr,x):
#   for i in range(len(arr)-1):
#     if arr[i] == x:
#       return i
#     else:
#       return -1

# arr = [int(i) for i in dataset2['E Code']]

# for i in range(len(df)-1):
#   # Function call
#   result = linearSearch(arr,  df.iloc[i]['E.Code'])

#   if result != -1:
#       ll1.append([dataset2.iloc[result]['Designation'],dataset2.iloc[result]['Operation'],dataset2.iloc[result]['Division'],dataset2.iloc[result]['Department']])
#   else:
#     print("Element is not present in array")

In [66]:
# Converting 2d list to dataframe to apply CRUD opertations
dff = pd.DataFrame(ll1, columns=['Designation','Operation','Division','Department'])

In [67]:
df_final = pd.concat([df['E.Code'], dff, df[['Date','OT']]], axis=1)

In [68]:
""" Printing dataframe (E.Code, Designation, Opr., Division, Dept., Date, OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes)) 
"""
df_final

,E.Code,Designation,Operation,Division,Department,Date,OT
0,174,Manager,Manesar Plant,Production Body Manesar,Paint,2022-10-01,0.17
1,189,Manager,Tapukara Plant,Plant Engineering Tapukara,Maintenance Tapukara,2022-10-01,0.04
2,361,Staff,Manesar Plant,Production Body Manesar,Assembly Frame,2022-10-01,1.56
3,368,General Manager,Narsapura Plant,Production Control Narsapura,NaN,2022-10-01,0.29
4,442,Chief Manager,Manesar Plant,Plant Engineering Manesar,Safety & Environment,2022-10-01,0.36
...,...,...,...,...,...,...,...
4195,30010145,Apprentice,Narsapura Plant,Production Body Narsapura,Weld Narsapura,2022-10-08,1.20
4196,30010146,Apprentice,Narsapura Plant,Production Body Narsapura,Weld Narsapura,2022-10-08,1.20
4197,30010147,Apprentice,Narsapura Plant,Production Power Train Narsapura,Machining 1 P/T,2022-10-08,0.04
4198,30010148,Apprentice,Narsapura Plant,Production Power Train Narsapura,Machining 2 P/T,2022-10-08,1.09


In [69]:
# df_final.iloc[10035]

In [70]:
""" Applying Groupby opr. to group dataframe on basis of E.Code and apply sum and count opr. to 
compute Total_OT & Total_OT_Days """

groupby_ecode_Total_OT = df.groupby(['E.Code']).sum()
groupby_ecode_Total_OT_Days = df.groupby(['E.Code']).count()
groupby_operation_sum = df_final.groupby(['Operation']).sum()
groupby_operation = df_final.groupby(['Operation']).count()
groupby_division_sum = df_final.groupby(['Division']).sum()
groupby_division = df_final.groupby(['Division']).count()
groupby_designation_sum = df_final.groupby(['Designation']).sum()
groupby_designation = df_final.groupby(['Designation']).count()

In [71]:
#Converting Groupby object into DataFrame to apply CRUD opr.
df2=pd.DataFrame(groupby_ecode_Total_OT)
df3=pd.DataFrame(groupby_ecode_Total_OT_Days)
df4=pd.DataFrame(groupby_operation)
df5=pd.DataFrame(groupby_division)
df6=pd.DataFrame(groupby_operation_sum)
df7=pd.DataFrame(groupby_division_sum)
df8=pd.DataFrame(groupby_designation)
df9=pd.DataFrame(groupby_designation_sum)

In [72]:
df4=df4.drop(['E.Code','Designation','Division','Department','Date'], axis=1)
df5=df5.drop(['E.Code','Designation','Operation','Department','Date'], axis=1)
df8=df8.drop(['E.Code','Operation','Division','Department','Date'], axis=1)
df6=df6.drop(['E.Code'], axis=1)
df7=df7.drop(['E.Code'], axis=1)
df9=df9.drop(['E.Code'], axis=1)

In [73]:
df4.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df5.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df8.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df6.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df7.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [74]:
l4=[]
for i in df2.index:
      frac, whole = math.modf(df2['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      l4.append(ot_time)

In [75]:
llp=[]
for i in df6.index:
      frac, whole = math.modf(df6['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      llp.append(ot_time)

In [76]:
lllp=[]
for i in df7.index:
      frac, whole = math.modf(df7['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      lllp.append(ot_time)

In [77]:
llllp=[]
for i in df9.index:
      frac, whole = math.modf(df9['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      llllp.append(ot_time)

In [78]:
df2['Total OT']=l4
df6['Total OT']=llp
df7['Total OT']=lllp
df9['Total OT']=llllp

In [79]:
#Droping an unnecesary column
df3=df3.drop(['Date'], axis=1)

In [80]:
df2=df2.drop(['OT'], axis=1)
df6=df6.drop(['OT'], axis=1)
df7=df7.drop(['OT'], axis=1)
df9=df9.drop(['OT'], axis=1)

In [81]:
#Renaming Columns
#df2.rename(columns={'OT': 'Total OT'}, inplace=True)
df3.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [82]:
#Creating a final dataframe (E.Code, Total OT_Days, Total OT)
result = pd.concat([df3,df2], axis=1)
result_2 =  pd.concat([df4,df6], axis=1)
result_3 =  pd.concat([df5,df7], axis=1)
result_4 =  pd.concat([df8,df9], axis=1)

In [83]:
""" Printing dataframe (E.Code, Total OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes), 
    Total OT_Days) 
"""
result

,Total OT_Days,Total OT
E.Code,,
32,1,0.09
49,1,0.13
72,1,0.02
80,3,2.36
88,1,0.39
...,...,...
30010147,3,0.14
30010148,4,3.31
30010149,2,0.18


In [84]:
# Sorting dataframe in descending order on basis of Total OT_Days (User can get top x employees on basis of Total OT_Days)
cont2=result.sort_values(by=['Total OT_Days'], ascending=False)

In [85]:
# Sorting dataframe in descending order on basis of Total OT (User can get top x employees on basis of Total OT)
cont3=result.sort_values(by=['Total OT'], ascending=False)

In [86]:
cont4=result_2.sort_values(by=['Total OT_Days'], ascending=False)

In [87]:
cont5=result_2.sort_values(by=['Total OT'], ascending=False)

In [88]:
cont6=result_3.sort_values(by=['Total OT_Days'], ascending=False)

In [89]:
cont7=result_3.sort_values(by=['Total OT'], ascending=False)

In [90]:
cont8=result_4.sort_values(by=['Total OT_Days'], ascending=False)

In [91]:
cont9=result_4.sort_values(by=['Total OT'], ascending=False)

In [92]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
from openpyxl import Workbook
import xlsxwriter
import os

workbook = xlsxwriter.Workbook(r'OT_Analysis.xlsx')
with pd.ExcelWriter(r'OT_Analysis.xlsx') as writer:
#writer = r"C:\Users\Kartikey\Desktop\shorttt.xlsx"
# write dataframe to excel
    df_final.to_excel(writer,sheet_name='OT')
    result.to_excel(writer,sheet_name='Total_OT')
    cont2.to_excel(writer,sheet_name='Descending_Total_OT_Days')
    cont3.to_excel(writer,sheet_name='Descending_Total_OT')
    cont4.to_excel(writer,sheet_name='Operation_Highest_Total_OT_Days')
    cont5.to_excel(writer,sheet_name='Operation_Highest_Total_OT')
    cont6.to_excel(writer,sheet_name='Division_Highest_Total_OT_Days')
    cont7.to_excel(writer,sheet_name='Division_Highest_Total_OT')
    cont8.to_excel(writer,sheet_name='Desig_Highest_Total_OT_Days')
    cont9.to_excel(writer,sheet_name='Desig_Highest_Total_OT')